# Import Data/Pkg, Process Data

In [1]:
using Pkg, CategoricalArrays, CSV, DataFrames, Statistics
using StatsBase

In [2]:
data = CSV.read("train.csv", DataFrame);

In [3]:
df = data;

In [4]:
df.Vehicle_Damage = map(x -> x == "Yes" ? 1 : 0, df.Vehicle_Damage);
df.Vehicle_Age = map(x -> x == "> 2 Years" ? 2 : x == "1-2 Year" ? 1 : 0, df.Vehicle_Age);
df[!, "Gender"] = categorical(df[!, "Gender"])
df[!, "Annual_Premium"] = df[!, "Annual_Premium"].*0.012;

In [5]:
t_options = 100:100:1000

100:100:1000

## Split Data

In [6]:
seed = 12345; 
X = df[:, Not([:Response, :Annual_Premium, :id, :Region_Code, :Policy_Sales_Channel])]
y = df.Response
t = df.Annual_Premium
(train_X, train_t, train_y), (test_X, test_t, test_y) = IAI.split_data(
    :policy_maximize, X, t, y, train_proportion=0.5, seed=seed);

In [7]:
train_t_discrete = [min(round(x, digits=-2), 1000) < 100 ? 100 : min(round(x, digits=-2), 1000) for x in train_t]
test_t_discrete = [min(round(x, digits=-2), 1000) < 100 ? 100 : min(round(x, digits=-2), 1000) for x in test_t];

## Prescriptive Tree

In [14]:
prescriptive_grid = IAI.GridSearch(
    IAI.OptimalTreePrescriptionMaximizer(
      random_seed=12345,
    ),
    max_depth=1:6,
)
# multiple the t(price)revenue with binary var y(1,0) buy or not buy
train_y_revenue = train_y .* train_t_discrete;

In [ ]:
IAI.fit!(prescriptive_grid, train_X, train_t_discrete, train_y_revenue)

In [11]:
pred_treatments, pred_outcomes = IAI.predict(prescriptive_grid, test_X)

(Real[900.0, 900.0, 800.0, 600.0, 900.0, 900.0, 600.0, 800.0, 1000.0, 1000.0  …  1000.0, 800.0, 1000.0, 1000.0, 1000.0, 1000.0, 600.0, 1000.0, 800.0, 1000.0], [771.4285714285713, 900.0, 8.333333333333258, 0.4329004329003965, 450.0, 242.30769230769215, 0.4329004329003965, 1.512287334593566, 369.69696969696975, 361.70212765957444  …  369.69696969696975, 1.512287334593566, 369.69696969696975, 666.6666666666667, 111.1111111111112, 666.6666666666667, 0.4329004329003965, 369.69696969696975, 299.9999999999999, 111.1111111111112])

In [13]:
df_w_prescri = hcat(test_X, pred_treatments);
CSV.write("./prescription.csv", df_w_prescri)

"./prescription.csv"

### Prescriptive Tree Evaulation

## Real Data

In [22]:
# Get revenue observed for test set in reality
test_revenue = test_y .* test_t;
print("The real life data suggest the annual revenue would be ", sum(test_revenue))
print("\n")

test_revenue_cut = test_y .* test_t_discrete;
print("If we do the cutoff on real life data, the annual revenue would be ", sum(test_revenue_cut))

The real life data suggest the annual revenue would be 8.851845936e6
If we do the cutoff on real life data, the annual revenue would be 9.0881e6

## Our Prescription

In [11]:
reward_lnr = IAI.NumericClassificationRewardEstimator(
    outcome_estimator=IAI.XGBoostClassifier(num_round=10),
    outcome_insample_num_folds=2,
    reward_estimator=:direct_method,
    estimation_kernel_bandwidth=1,
    random_seed=seed,
)

function get_rewards(reward_lnr, X, t, y, t_options)
  predictions, score = IAI.fit_predict!(reward_lnr, X, t, y, t_options,
                                        outcome_score_criterion=:auc)
  rewards = predictions[:reward]
  for t in t_options
    rewards[!, Symbol(t)] = round.(rewards[!, Symbol(t)] .* t, digits=3)
  end
  rewards, score
end
train_rewards, train_reward_score = get_rewards(reward_lnr, train_X, train_t_discrete,
                                                train_y, t_options)
train_rewards

Row,100,200,300,400,500,600,700,800,900,1000
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.101,8.842,14.141,10.759,14.063,20.654,15.839,25.963,28.692,29.657
2,14.094,22.194,27.6,54.552,70.734,69.247,77.908,49.112,46.613,90.961
3,2.278,4.522,6.737,9.259,11.383,13.723,16.175,19.322,22.002,23.923
4,2.278,4.522,6.737,9.259,11.383,13.723,16.175,19.322,22.002,23.923
5,28.416,47.233,86.22,126.352,169.403,195.783,267.792,207.891,280.271,424.877
6,2.274,4.586,6.769,9.159,11.161,13.658,15.839,19.812,21.535,23.377
7,25.744,21.061,58.848,62.675,97.485,192.127,141.875,92.315,523.358,81.112
8,12.288,13.36,41.945,60.443,58.114,76.321,111.072,138.247,146.848,126.628
9,2.278,4.522,6.737,9.259,11.383,13.723,16.175,19.322,65.753,23.923


In [12]:
train_reward_score[:outcome]

Dict{String, Float64} with 10 entries:
  "300"  => 0.881468
  "400"  => 0.852542
  "1000" => 0.771416
  "600"  => 0.813256
  "800"  => 0.814287
  "700"  => 0.804718
  "100"  => 0.78973
  "200"  => 0.898189
  "500"  => 0.831272
  "900"  => 0.728133

In [13]:
test_rewards, test_reward_score = get_rewards(reward_lnr, test_X, test_t_discrete,
                                              test_y, t_options)
test_rewards

Row,100,200,300,400,500,600,700,800,900,1000
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,26.087,51.986,95.504,142.665,160.878,245.609,268.517,387.959,149.301,400.693
2,27.092,63.244,97.994,141.605,168.236,261.982,281.876,433.982,504.174,451.658
3,2.308,4.651,6.751,9.095,11.448,13.49,15.929,18.607,21.866,23.759
4,2.308,4.651,6.751,9.095,11.448,13.49,15.929,18.607,21.866,23.759
5,11.227,32.144,40.833,52.682,71.985,103.195,94.231,233.867,50.961,104.734
6,21.522,57.827,81.104,84.874,119.565,144.296,174.057,168.949,90.098,165.119
7,2.298,4.531,6.817,9.046,11.291,13.788,16.46,18.82,23.496,24.011
8,2.308,4.651,6.751,9.095,11.448,13.49,15.929,18.607,21.866,23.759
9,29.865,84.643,99.11,139.354,178.188,191.524,225.038,307.405,263.512,281.059


In [14]:
test_reward_score[:outcome]

Dict{String, Float64} with 10 entries:
  "300"  => 0.880268
  "400"  => 0.851357
  "1000" => 0.799334
  "600"  => 0.81237
  "800"  => 0.787536
  "700"  => 0.800373
  "100"  => 0.78816
  "200"  => 0.89275
  "500"  => 0.827358
  "900"  => 0.768116

In [15]:
function evaluate(recommendations, outcomes, actual_revenue)
  n = length(recommendations)
  pred_revenue = [outcomes[i, recommendations[i]] for i in 1:n]

  improvement = mean(pred_revenue .- actual_revenue) / mean(actual_revenue)
end

evaluate (generic function with 1 method)

In [16]:
function evaluate2(recommendations, outcomes, actual_revenue)
  n = length(recommendations)
  pred_revenue = [outcomes[i, recommendations[i]] for i in 1:n]

  tot_rev = sum(pred_revenue)
end

evaluate2 (generic function with 1 method)

In [17]:
test_revenue = test_y .* test_t_discrete;

In [18]:
recommendations_pres = Symbol.(Int.(IAI.predict(prescriptive_grid, test_X)[1]))

190555-element Vector{Symbol}:
 Symbol("900")
 Symbol("900")
 Symbol("800")
 Symbol("600")
 Symbol("900")
 Symbol("900")
 Symbol("600")
 Symbol("800")
 Symbol("1000")
 Symbol("1000")
 Symbol("900")
 Symbol("800")
 Symbol("900")
 ⋮
 Symbol("800")
 Symbol("1000")
 Symbol("1000")
 Symbol("800")
 Symbol("1000")
 Symbol("1000")
 Symbol("1000")
 Symbol("1000")
 Symbol("600")
 Symbol("1000")
 Symbol("800")
 Symbol("1000")

In [19]:
evaluate(recommendations_pres, test_rewards, test_revenue)

2.014074493458478

In [20]:
evaluate2(recommendations_pres, test_rewards, test_revenue)

2.7392210404000007e7